In [1]:
import pandas as pd
# Đường dẫn đến tệp Excel
file_path = 'D:\Báo cáo tốt nghiệp\Check.xlsx'

# Đọc dữ liệu từ tệp Excel
df = pd.read_excel(file_path)
print(df)

<>:3: SyntaxWarning: invalid escape sequence '\B'
<>:3: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\1294996154.py:3: SyntaxWarning: invalid escape sequence '\B'
  file_path = 'D:\Báo cáo tốt nghiệp\Check.xlsx'
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\1294996154.py:3: SyntaxWarning: invalid escape sequence '\B'
  file_path = 'D:\Báo cáo tốt nghiệp\Check.xlsx'


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Báo cáo tốt nghiệp\\Check.xlsx'

In [ ]:
# # Lặp qua từng sinh viên
# for msv in df['MSV'].unique():
#     # Lọc dữ liệu cho sinh viên cụ thể
#     student_data = df[df['MSV'] == msv]
#     # Danh sách các học phần của sinh viên
#     student_courses = student_data['Mã học phần'].unique()
    
#     # Khởi tạo một từ điển để lưu thông tin ca thi cho mỗi học phần
#     exam_sessions_dict = {}
    
#     # Lặp qua từng học phần của sinh viên
#     for mhp in student_courses:
#         # Lọc dữ liệu cho học phần cụ thể
#         course_data = student_data[student_data['Mã học phần'] == mhp]
#         # Danh sách các ca thi của học phần
#         exam_sessions = course_data['Ca thi'].unique()
        
#         # Chuyển đổi các giá trị thành chuỗi trước khi sử dụng join
#         exam_sessions_str = [str(session) for session in exam_sessions]
        
#         # Lưu thông tin ca thi vào từ điển
#         exam_sessions_dict[mhp] = ', '.join(exam_sessions_str)
    
#     # In thông tin về các học phần và ca thi của sinh viên
#     courses_and_sessions = [f"{course}: [{exam_sessions_dict[course]}]" for course in exam_sessions_dict]
#     print(f"Sinh viên {msv} tham gia học phần {courses_and_sessions}")


In [3]:
import numpy
import pandas as pd
import math

def assign_exam_schedule(df, max_shifts, max_rooms, students_per_room):
    course_students = df.groupby('Mã học phần')['MSV'].count()
    course_rooms = dict([[kcs, math.ceil(course_students[kcs]/students_per_room)] for kcs in course_students.keys()])
    #Xem mã học phần đó có bao nhiêu sinh viên đăng ký
    count_node = df.groupby('Mã học phần')['MSV'].nunique().reset_index(name='SLSV')
    students_per_shift = students_per_room*max_rooms
    shifts = numpy.zeros(max_shifts)
    df['Ca thi'] = numpy.nan
    # Sắp xếp các mã học phần theo slsv đăng ký
    sorted_nodes = count_node.sort_values(by='SLSV', ascending=False)['Mã học phần'].tolist()     
    #Tạo các cạnh nối giữa các mã học phần
    edges = set()

    # Xem sinh viên đó học bao nhiêu học phần 
    for _, group in df.groupby('MSV'): 
        #       
        subjects = group['Mã học phần'].tolist()
        
        #Tạo ra các cặp cạnh i,j trong danh sách subject. Các cặp này sẽ tạo thành các cạnh biễu diễn bằng giá trị i,j
        edges.update([(subjects[i], subjects[j]) for i in range(len(subjects)) for j in range(i+1, len(subjects))])

    i = 1    
    while sorted_nodes and i <= max_shifts:
        # Chọn đỉnh đầu tiên trong danh sách V
        start_node = sorted_nodes[0]
        
        # Tô màu i cho đỉnh đang xét
        #Số lượng sinh viên của mã học phần đầu tiên
        no_students = len(df.loc[df['Mã học phần'] == start_node, 'Ca thi'])
        #no_students = course_students[start_node]
        if no_students > students_per_shift:
            if shifts[i] == 0:
                no_required_shifts = math.ceil(no_students/students_per_shift)
                no_node_students_per_shift = math.ceil(no_students/no_required_shifts)
                j = 0
                for k,r in df.loc[df['Mã học phần'] == start_node].iterrows():
                    df.loc[k,'Ca thi'] = i+int(j/no_node_students_per_shift)
                    j += 1
                
                for j in range(no_required_shifts):
                    shifts[i+j] += math.ceil(no_node_students_per_shift/students_per_room)
                    
                sorted_nodes = sorted_nodes[1:]
                
        elif no_students < students_per_shift:
            #Lọc ra các học phần còn lại
            unassigned_course = df[df['Ca thi'].isnull()]['Mã học phần'].unique()
            new_edges = set()   
        
            for _, group in df[df['Mã học phần'].isin(unassigned_course)].groupby('MSV'):
            # Tạo danh sách chứa các mã học phần trong nhóm hiện tại
                subjects = group['Mã học phần'].tolist()
            # Tạo các cặp cạnh giữa các mã học phần trong nhóm
                new_edges.update([(subjects[i], subjects[j]) for i in range(len(subjects)) for j in range(i + 1, len(subjects))])
            # Tạo từ điển để lưu trữ số cạnh của mỗi node
            node_edge_counts = {}
            #if 'ENG3008' in node_edge_counts:
            #print(node_edge_counts['ENG3008'])
            for edge in new_edges:
                for node in edge:
                    # Tăng số lượng cạnh cho node lên 1
                    if node in node_edge_counts:
                        node_edge_counts[node] += 1
                    else:       
                        node_edge_counts[node] = 1
            # Sắp xếp các cạnh của mỗi node theo thứ tự giảm dần
            sorted_nodes_by_edge_count = sorted(node_edge_counts.keys(), key=lambda x: node_edge_counts[x], reverse=True)
            if len(sorted_nodes_by_edge_count)>0:   
                first_node=sorted_nodes_by_edge_count[0]
               
                df.loc[df['Mã học phần'] == first_node, 'Ca thi'] = i
                
                sorted_nodes_by_edge_count = sorted_nodes_by_edge_count[1:]

                # sorted_nodes = sorted_nodes[1:]
             
        jn = 0
        while jn < len(sorted_nodes):
            node =  sorted_nodes[jn]
            #Danh sách các đỉnh đã được tô màu i
            colored_nodes = set(df[df['Ca thi'] == i]['Mã học phần'].tolist())
            if all((node, colored_node) not in edges and (colored_node, node) not in edges for colored_node in colored_nodes):
                # Nếu không kề với các học phần đã tô màu i, tô màu i cho đỉnh đó
                remaining_room=max_rooms-shifts[i]
                if course_rooms[node] <= remaining_room:                
                    df.loc[df['Mã học phần'] == node, 'Ca thi'] = i
                    shifts[i] += course_rooms[node]
                    #uncolored_nodes.remove(node)
                    sorted_nodes = sorted_nodes[:jn] + sorted_nodes[jn+1:]
                    #colored_nodes += [node]
                    jn -= 1
            else:
                print(node)
            jn += 1
     
        # Tăng giá trị màu i lên 1 đơn vị
        i += 1
# Gọi hàm để gán ca thi
MAX_SHIFTS = 60
MAX_ROOMS = 35
STUDENTS_PER_ROOM = 40
assign_exam_schedule(df, MAX_SHIFTS, MAX_ROOMS, STUDENTS_PER_ROOM)

Mã học phần
ACC1001          24
ACC1001_Anh       1
ACC1002        1692
ACC1002_Anh     222
ACC1902_Anh       7
               ... 
TOU3056          99
TOU3060          55
VIS1909          49
VNA1002          13
VNA2004          13
Name: MSV, Length: 256, dtype: int64
     Mã học phần   Bậc
0        ACC1001    24
1    ACC1001_Anh     1
2        ACC1002  1692
3    ACC1002_Anh   222
4    ACC1902_Anh     7
..           ...   ...
251      TOU3056    99
252      TOU3060    55
253      VIS1909    49
254      VNA1002    13
255      VNA2004    13

[256 rows x 2 columns]
SMT1007
TOU1001
MIS2002
ACC1002
ENG2015
SMT1004
SMT1008
ENG3001
SMT1006
FIN2001
HRM2001
IBS2001
MGT2002
LAW2001
ACC3004
FIN3004
MGT3004
HOS3001
ENGELE2
ACC3001
FIN3002
MIS2001
FIN3001
FIN2001_Anh
MGT3002
HRM3005
IBS3001
LAW3001
AUD3004
ACC2004
ENG3005
IBS3007
BAN3014
STA3001
ACC3008
MGT3007
ACC3002
ENG2016
ENG2017
FIN3003
ECO3001
ACC1002_Anh
MGT1002
ECO2001
BAN2001
MIS3006
IBS3013
IBS3004
SMT3010
BAN3001
FIN3004_Anh
LAW2007
ECO

In [4]:
# Nhóm dữ liệu theo cột 'MSV' và 'Ca thi', sau đó đếm số lượng dòng trong mỗi nhóm
duplicate_schedule = df.groupby(['MSV', 'Ca thi']).size().reset_index(name='Count')

# Lọc ra các nhóm có số lượng dòng lớn hơn 1, tức là các MSV bị trùng lịch thi
duplicated_students = duplicate_schedule[duplicate_schedule['Count'] > 1]   

if not duplicated_students.empty:
    print("Có MSV bị trùng lịch thi:")
    print(duplicated_students)
else:
    print("Không có MSV nào bị trùng lịch thi.")

Có MSV bị trùng lịch thi:
        MSV  Ca thi  Count
29490  6124    38.0      2


In [3]:
df.to_excel('D:\Báo cáo tốt nghiệp\Check.xlsx', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\B'
<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\Admin\AppData\Local\Temp\ipykernel_16232\3263794764.py:1: SyntaxWarning: invalid escape sequence '\B'
  df.to_excel('D:\Báo cáo tốt nghiệp\Check.xlsx', index=False)
